In [1]:
import requests
import json
import pandas as pd
from datetime import datetime

#Use this code to collect a snapshot of the details of the EVSEs in Switzerland at a single point in time.
#The output is a .csv saved to the file_path defined in the last block.

In [2]:
# URL to the JSON file (choose English or default)
url = "https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/data/ch.bfe.ladestellen-elektromobilitaet.json"
response = requests.get(url)
response.raise_for_status()
data = response.json()

# Print top-level keys
print("Top-level keys:", list(data.keys()))

# Print the number of EVSE blocks
print("Number of 'EVSEData' items:", len(data["EVSEData"]))

# Print first EVSEData block with limited nested keys
sample_block = data["EVSEData"][0]
sample_record = sample_block.get("EVSEDataRecord", [{}])[0]

# Limit the fields we print to avoid overload
fields_to_print = {
    "EvseID": sample_record.get("EvseID"),
    "ChargingStationId": sample_record.get("ChargingStationId"),
    "City": sample_record.get("Address", {}).get("City"),
    "Country": sample_record.get("Address", {}).get("Country"),
    "GeoCoordinates": sample_record.get("GeoCoordinates", {}).get("Google"),
    "ChargingFacilities": sample_record.get("ChargingFacilities"),
}

print("\nSample EVSE record (limited fields):")
print(json.dumps(fields_to_print, indent=2))


Top-level keys: ['EVSEData']
Number of 'EVSEData' items: 36

Sample EVSE record (limited fields):
{
  "EvseID": "CH*SWI*E10382",
  "ChargingStationId": "CH*SWI*E10382",
  "City": "Lostorf",
  "Country": "CHE",
  "GeoCoordinates": "47.37545935 7.94404536",
  "ChargingFacilities": [
    {
      "power": 20,
      "powertype": "DC"
    },
    {
      "power": 20,
      "powertype": "DC"
    }
  ]
}


In [3]:
# Load data
# url = "https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/data/ch.bfe.ladestellen-elektromobilitaet_en.json"
url = "https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/data/ch.bfe.ladestellen-elektromobilitaet.json"
response = requests.get(url)
response.raise_for_status()
data = response.json()

# Initialize records list
records = []

# Go into "EVSEData" → each has an "EVSEDataRecord" list
for container in data.get("EVSEData", []):
    for item in container.get("EVSEDataRecord", []):
        address = item.get("Address", {})
        geo = item.get("GeoCoordinates", {}).get("Google", None)
        facilities = item.get("ChargingFacilities", [])

        record_base = {
            "EvseID": item.get("EvseID"),
            "ChargingStationId": item.get("ChargingStationId"),
            "Accessibility": item.get("Accessibility"),
            "GeoCoordinates": geo,
            "City": address.get("City"),
            "Country": address.get("Country"),
            "PostalCode": address.get("PostalCode"),
            "Street": address.get("Street"),
            "ParkingSpot": address.get("ParkingSpot"),
            "ParkingFacility": address.get("ParkingFacility"),
            # "ChargingFacilities": facilities,
            # "ChargingFacility_power": power,
            # "ChargingFacility_powertype": powertype,
            "DynamicInfoAvailable": item.get("DynamicInfoAvailable"),
            "IsOpen24Hours": item.get("IsOpen24Hours"),
            "LastUpdate": item.get("lastUpdate"),
            "MaxCapacity": item.get("MaxCapacity"),
            "DynamicPowerLevel": item.get("DynamicPowerLevel"),
            "EnergySource": item.get("EnergySource")
        }
        # If no charging facilities, add one record with None for power and powertype
        if not facilities:
            record = record_base.copy()
            record["ChargingFacility_power"] = None
            record["ChargingFacility_powertype"] = None
            records.append(record)
        else:
            for fac in facilities:
                record = record_base.copy()
                record["ChargingFacility_power"] = fac.get("power")
                record["ChargingFacility_powertype"] = fac.get("powertype")
                records.append(record)

        # records.append(record)

# Create DataFrame
df = pd.DataFrame(records)
pd.set_option("display.max_columns", None)
print(df.head(10))


           EvseID ChargingStationId             Accessibility  \
0   CH*SWI*E10382     CH*SWI*E10382  Free publicly accessible   
1   CH*SWI*E10382     CH*SWI*E10382  Free publicly accessible   
2   CH*SWI*E10383     CH*SWI*E10382  Free publicly accessible   
3  CH*SWI*E262873     CH*SWI*E10382  Free publicly accessible   
4  CH*SWI*E262874     CH*SWI*E10382  Free publicly accessible   
5  CH*SWI*E262875     CH*SWI*E10382  Free publicly accessible   
6  CH*SWI*E262876     CH*SWI*E10382  Free publicly accessible   
7  CH*SWI*E262877     CH*SWI*E10382  Free publicly accessible   
8  CH*SWI*E262878     CH*SWI*E10382  Free publicly accessible   
9  CH*SWI*E262879     CH*SWI*E10382  Free publicly accessible   

           GeoCoordinates     City Country PostalCode          Street  \
0  47.37545935 7.94404536  Lostorf     CHE       4654  Grundstrasse 1   
1  47.37545935 7.94404536  Lostorf     CHE       4654  Grundstrasse 1   
2  47.37545935 7.94404536  Lostorf     CHE       4654  Grundstras

In [4]:
# URL for the English JSON dataset (GeoJSON format)
url = "https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/data/ch.bfe.ladestellen-elektromobilitaet_en.json"
response = requests.get(url)
response.raise_for_status()
data = response.json()

# Print top-level keys
print("Top-level keys:", list(data.keys()))

# Number of features (charging points)
print("Number of features:", len(data.get("features", [])))

# Sample first feature
sample_feature = data["features"][0]

# Limit fields to print from this feature
fields_to_print = {
    "id": sample_feature.get("id"),
    "type": sample_feature.get("type"),
    "coordinates": sample_feature.get("geometry", {}).get("coordinates"),
    "properties": {
        "location_id": sample_feature.get("properties", {}).get("location_id"),
        "Availability": sample_feature.get("properties", {}).get("Availability"),
        "description_snippet": sample_feature.get("properties", {}).get("description", "")[:200]  # first 200 chars
    }
}

print("\nSample feature (limited fields):")
print(json.dumps(fields_to_print, indent=2))


Top-level keys: ['type', 'name', 'comment', 'crs', 'features']
Number of features: 7936

Sample feature (limited fields):
{
  "id": "CH*SWI*E10382",
  "type": "Feature",
  "coordinates": [
    7.94404536,
    47.37545935
  ],
  "properties": {
    "location_id": "CH*SWI*E10382",
    "Availability": "OutOfService",
    "description_snippet": "<div class=\"evse-data\">\n    \n    <table class=\"evse-overview status-outofservice\">\n        <tr>\n            <th>Out of service</th>\n        </tr>\n        \n            <tr>\n                <td>\n       "
  }
}


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17340 entries, 0 to 17339
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   EvseID                      17340 non-null  object 
 1   ChargingStationId           17340 non-null  object 
 2   Accessibility               17340 non-null  object 
 3   GeoCoordinates              17340 non-null  object 
 4   City                        17340 non-null  object 
 5   Country                     17340 non-null  object 
 6   PostalCode                  16985 non-null  object 
 7   Street                      17340 non-null  object 
 8   ParkingSpot                 0 non-null      object 
 9   ParkingFacility             5973 non-null   object 
 10  DynamicInfoAvailable        17340 non-null  object 
 11  IsOpen24Hours               17340 non-null  object 
 12  LastUpdate                  4933 non-null   object 
 13  MaxCapacity                 590

In [6]:
df.head(10)

,EvseID,ChargingStationId,Accessibility,GeoCoordinates,City,Country,PostalCode,Street,ParkingSpot,ParkingFacility,DynamicInfoAvailable,IsOpen24Hours,LastUpdate,MaxCapacity,DynamicPowerLevel,EnergySource,ChargingFacility_power,ChargingFacility_powertype
0,CH*SWI*E10382,CH*SWI*E10382,Free publicly accessible,47.37545935 7.94404536,Lostorf,CHE,4654,Grundstrasse 1,None,None,true,True,2025-09-30T02:15:16.965Z,NaN,None,None,20,DC
1,CH*SWI*E10382,CH*SWI*E10382,Free publicly accessible,47.37545935 7.94404536,Lostorf,CHE,4654,Grundstrasse 1,None,None,true,True,2025-09-30T02:15:16.965Z,NaN,None,None,20,DC
2,CH*SWI*E10383,CH*SWI*E10382,Free publicly accessible,47.37545935 7.94404536,Lostorf,CHE,4654,Grundstrasse 1,None,None,true,True,2025-09-30T02:15:16.965Z,NaN,None,None,22,AC_3_PHASE
3,CH*SWI*E262873,CH*SWI*E10382,Free publicly accessible,47.37545935 7.94404536,Lostorf,CHE,4654,Grundstrasse 1,None,None,true,True,2025-09-30T02:15:16.965Z,NaN,None,None,None,None
4,CH*SWI*E262874,CH*SWI*E10382,Free publicly accessible,47.37545935 7.94404536,Lostorf,CHE,4654,Grundstrasse 1,None,None,true,True,2025-09-30T02:15:16.965Z,NaN,None,None,None,None
5,CH*SWI*E262875,CH*SWI*E10382,Free publicly accessible,47.37545935 7.94404536,Lostorf,CHE,4654,Grundstrasse 1,None,None,true,True,2025-09-30T02:15:16.965Z,NaN,None,None,None,None
6,CH*SWI*E262876,CH*SWI*E10382,Free publicly accessible,47.37545935 7.94404536,Lostorf,CHE,4654,Grundstrasse 1,None,None,true,True,2025-09-30T02:15:16.965Z,NaN,None,None,None,None
7,CH*SWI*E262877,CH*SWI*E10382,Free publicly accessible,47.37545935 7.94404536,Lostorf,CHE,4654,Grundstrasse 1,None,None,true,True,2025-09-30T02:15:16.965Z,NaN,None,None,None,None
8,CH*SWI*E262878,CH*SWI*E10382,Free publicly accessible,47.37545935 7.94404536,Lostorf,CHE,4654,Grundstrasse 1,None,None,true,True,2025-09-30T02:15:16.965Z,NaN,None,None,None,None
9,CH*SWI*E262879,CH*SWI*E10382,Free publicly accessible,47.37545935 7.94404536,Lostorf,CHE,4654,Grundstrasse 1,None,None,true,True,2025-09-30T02:15:16.965Z,NaN,None,None,None,None


In [12]:
# Format today's date
today_str = datetime.today().strftime("%d-%m-%Y")

# File path
file_path = f"M:/Rudolph/R-Projects/swiss-public-charging/data/ChargingStationDetails_{today_str}.csv"

# Export to CSV
df.to_csv(file_path, index=False)
